In [1]:
from tweepy import AppAuthHandler, TweepError
from tweepy.models import User as TweepyUser, Status
import tweepy
import pandas as pd
from tqdm import tqdm
from dask.distributed import Client
from dask import dataframe as dd

In [2]:
client = Client()

2022-04-05 08:25:38,147 - distributed.diskutils - INFO - Found stale lock file and directory '/home/ubuntu/argminer/Experiments/dask-worker-space/worker-l1xjdyvm', purging
2022-04-05 08:25:38,148 - distributed.diskutils - INFO - Found stale lock file and directory '/home/ubuntu/argminer/Experiments/dask-worker-space/worker-0vyrsajq', purging


In [3]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 2
Total threads: 2,Total memory: 3.83 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:40451,Workers: 2
Dashboard: http://127.0.0.1:8787/status,Total threads: 2
Started: Just now,Total memory: 3.83 GiB
Comm: tcp://127.0.0.1:43831,Total threads: 1
Dashboard: http://127.0.0.1:33063/status,Memory: 1.92 GiB
Nanny: tcp://127.0.0.1:46815,


In [4]:
consumer_key = "5pFpPxoMDcJpd6OaHyjfoWA1A"
consumer_secret = "akg4oPOUDkx0RDVwAtuwyNd8TbF7C9MHct5pD0gabQVFhllv0E"
access_token = "1241616154860240896-jhWik5da8WWSn4bg5fBN05EAK98dmR"
access_token_secret = "6gd0n8bOcNh6jxVOjjjUsJ4J4n5pPtvfMhj72DDswEHSl"
auth = AppAuthHandler(consumer_key=consumer_key, consumer_secret=consumer_secret)
api = tweepy.API(auth, wait_on_rate_limit=True,wait_on_rate_limit_notify=True)

In [5]:
def get_full_text(tweet_id):
    try:
        status = api.get_status(tweet_id, tweet_mode="extended")
        try:
            full_text = status.retweeted_status.full_text
        except AttributeError:  # Not a Retweet
            full_text = status.full_text
        return full_text
    except TweepError:
        pass

df = dd.read_csv("../Data/full_dataset_clean.tsv",delimiter='\t')

df.loc[df.lang=='en'].to_csv("../Data/EnglishTweets")

In [6]:
df = dd.read_csv("../Data/EnglishTweets/*",names=['id','tweet_id','date','time','lang','country'],
                 dtype={'country': 'object','id': 'float64','tweet_id': 'object',
                        "tweet_id":"object","time":"object","date":"object","lang":"object"})

In [7]:
df.head()

,id,tweet_id,date,time,lang,country
0,9.0,1213330173736738817,2020-01-04,05:23:50,en,NaN
1,18.0,1214195710301618178,2020-01-06,14:43:10,en,NaN
2,19.0,1214198629553950721,2020-01-06,14:54:46,en,NaN
3,31.0,1214917256414662656,2020-01-08,14:30:20,en,NaN
4,32.0,1214936881563082752,2020-01-08,15:48:19,en,NaN


In [8]:
df.tweet_id.to_csv("EngTweetIds/*.csv",index=False)

['/home/ubuntu/argminer/Experiments/EngTweetIds/000.csv',
 '/home/ubuntu/argminer/Experiments/EngTweetIds/001.csv',
 '/home/ubuntu/argminer/Experiments/EngTweetIds/002.csv',
 '/home/ubuntu/argminer/Experiments/EngTweetIds/003.csv',
 '/home/ubuntu/argminer/Experiments/EngTweetIds/004.csv',
 '/home/ubuntu/argminer/Experiments/EngTweetIds/005.csv',
 '/home/ubuntu/argminer/Experiments/EngTweetIds/006.csv',
 '/home/ubuntu/argminer/Experiments/EngTweetIds/007.csv',
 '/home/ubuntu/argminer/Experiments/EngTweetIds/008.csv',
 '/home/ubuntu/argminer/Experiments/EngTweetIds/009.csv',
 '/home/ubuntu/argminer/Experiments/EngTweetIds/010.csv',
 '/home/ubuntu/argminer/Experiments/EngTweetIds/011.csv',
 '/home/ubuntu/argminer/Experiments/EngTweetIds/012.csv',
 '/home/ubuntu/argminer/Experiments/EngTweetIds/013.csv',
 '/home/ubuntu/argminer/Experiments/EngTweetIds/014.csv',
 '/home/ubuntu/argminer/Experiments/EngTweetIds/015.csv',
 '/home/ubuntu/argminer/Experiments/EngTweetIds/016.csv',
 '/home/ubuntu

In [87]:
df['full_text'] = df.tweet_id.apply(get_full_text,meta=("full_text",str))

df.to_csv("../Data/HydratedEngTweets/*.csv",index=False)